# Introduction

The notebook is intended to perform a binary classification over the 'Response' label.

In [34]:
# Import Standard Modules
import pandas as pd

from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import precision_score, accuracy_score
from sklearn.linear_model import LogisticRegression

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

import xgboost as xgb

# Set Pandas Options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Read Data

In [3]:
# Read data
data = pd.read_csv('./../data/marketing_campaign_prepared.csv', encoding='latin1', sep=';')

# Data Pre-processing

## Features & Label Definition

The 'ID' column does not bring any useful information.

In [4]:
# Define x and y
X = data.drop(['ID', 'Response'], axis=1)
y = data['Response']

## Train & Test Split

Since the label is characterized by a strong imbalancing in the class distribution, we need to address it carefully:
1. Ensure that the training and test sets have the same proportions of the two classes
2. Oversample the minor class (i.e., randomly duplicate examples)
3. Undersample the major class (i.e., randomly delete examples)
4. Use several metrics (e.g., Accuracy, Precision, Recall, AUC)

Use StratifiedShuffleSplit. This cross-validation object is a merge of StratifiedKFold and ShuffleSplit, which returns stratified randomized folds. The folds are made by preserving the percentage of samples for each class.

Note: like the ShuffleSplit strategy, stratified random splits do not guarantee that all folds will be different, although this is still very likely for sizeable datasets.

In [5]:
# Standard train & test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Define the a Stratified K-fold Shuffle Splitter
stratified_kfold = StratifiedShuffleSplit(n_splits=10,
                                          test_size=.3, 
                                          random_state=0)

## Feature Selection

In [6]:
# Numerical Features
numerical_features = ['Year_Birth', 
                      'Income', 
                      'Recency', 
                      'MntWines', 
                      'MntFruits', 
                      'MntMeatProducts', 
                      'MntFishProducts', 
                      'MntSweetProducts', 
                      'MntGoldProds', 
                      'NumDealsPurchases', 
                      'NumWebPurchases', 
                      'NumCatalogPurchases', 
                      'NumStorePurchases', 
                      'NumWebVisitsMonth']

# Categorical Text Features
categorical_text_features = ['Education', 
                             'Marital_Status']

# Categorical Numerical Features
categorical_numerical_features = ['Kidhome', 
                                  'Teenhome', 
                                  'AcceptedCmp1', 
                                  'AcceptedCmp2', 
                                  'AcceptedCmp3', 
                                  'AcceptedCmp4', 
                                  'AcceptedCmp5', 
                                  'Complain', 
                                  'Dt_Customer_month', 
                                  'Dt_Customer_dayofweek']

## Data Standardization

Transform the individual features to look more or less like standard normally distributed data: Gaussian with zero mean and unit variance.

Keep in mind that tree-based methods are scale-invariant, so data standardization is not required.

Standardization has to go after training-test split. That's because, standardizing the whole dataset and then split, would introduce into the training set some information about the mean and std of the test set. Remember to standardize the test set with the same scaler trained on the training set. This would be addressed by constructing a pipeline with the scaler as a step.

In [35]:
# Define the ColumnTransformer
column_transformer = ColumnTransformer([
    ('numerical', StandardScaler(), numerical_features),
    ('categorical_text', OneHotEncoder(), categorical_text_features),
    ('categorical_numerical', 'passthrough', categorical_numerical_features)
], verbose_feature_names_out=True)

In [36]:
# Fit the ColumnTransformer
column_transformer.fit(X_train)

ColumnTransformer(transformers=[('numerical_transformation', StandardScaler(),
                                 ['Year_Birth', 'Income', 'Recency', 'MntWines',
                                  'MntFruits', 'MntMeatProducts',
                                  'MntFishProducts', 'MntSweetProducts',
                                  'MntGoldProds', 'NumDealsPurchases',
                                  'NumWebPurchases', 'NumCatalogPurchases',
                                  'NumStorePurchases', 'NumWebVisitsMonth']),
                                ('categorical_text_transformation',
                                 OneHotEncoder(),
                                 ['Education', 'Marital_Status']),
                                ('categorical_numerical_transformation',
                                 'passthrough',
                                 ['Kidhome', 'Teenhome', 'AcceptedCmp1',
                                  'AcceptedCmp2', 'AcceptedCmp3',
                               

In [37]:
column_transformer.get_feature_names_out()

array(['numerical_transformation__Year_Birth',
       'numerical_transformation__Income',
       'numerical_transformation__Recency',
       'numerical_transformation__MntWines',
       'numerical_transformation__MntFruits',
       'numerical_transformation__MntMeatProducts',
       'numerical_transformation__MntFishProducts',
       'numerical_transformation__MntSweetProducts',
       'numerical_transformation__MntGoldProds',
       'numerical_transformation__NumDealsPurchases',
       'numerical_transformation__NumWebPurchases',
       'numerical_transformation__NumCatalogPurchases',
       'numerical_transformation__NumStorePurchases',
       'numerical_transformation__NumWebVisitsMonth',
       'categorical_text_transformation__Education_2n Cycle',
       'categorical_text_transformation__Education_Basic',
       'categorical_text_transformation__Education_Graduation',
       'categorical_text_transformation__Education_Master',
       'categorical_text_transformation__Education_PhD

In [41]:
# Transform the columns
X_train_transformed = pd.DataFrame(column_transformer.transform(X_train), columns=column_transformer.get_feature_names_out())
X_test_transformed = pd.DataFrame(column_transformer.transform(X_test), columns=column_transformer.get_feature_names_out())

In [42]:
X_train_transformed.head()

,numerical_transformation__Year_Birth,numerical_transformation__Income,numerical_transformation__Recency,numerical_transformation__MntWines,numerical_transformation__MntFruits,numerical_transformation__MntMeatProducts,numerical_transformation__MntFishProducts,numerical_transformation__MntSweetProducts,numerical_transformation__MntGoldProds,numerical_transformation__NumDealsPurchases,numerical_transformation__NumWebPurchases,numerical_transformation__NumCatalogPurchases,numerical_transformation__NumStorePurchases,numerical_transformation__NumWebVisitsMonth,categorical_text_transformation__Education_2n Cycle,categorical_text_transformation__Education_Basic,categorical_text_transformation__Education_Graduation,categorical_text_transformation__Education_Master,categorical_text_transformation__Education_PhD,categorical_text_transformation__Marital_Status_Divorced,categorical_text_transformation__Marital_Status_Married,categorical_text_transformation__Marital_Status_Single,categorical_text_transformation__Marital_Status_Together,categorical_text_transformation__Marital_Status_Widow,categorical_numerical_transformation__Kidhome,categorical_numerical_transformation__Teenhome,categorical_numerical_transformation__AcceptedCmp1,categorical_numerical_transformation__AcceptedCmp2,categorical_numerical_transformation__AcceptedCmp3,categorical_numerical_transformation__AcceptedCmp4,categorical_numerical_transformation__AcceptedCmp5,categorical_numerical_transformation__Complain,categorical_numerical_transformation__Dt_Customer_month,categorical_numerical_transformation__Dt_Customer_dayofweek
0,-0.770141,-0.872431,1.068216,-0.696798,-0.101041,-0.622485,-0.572726,0.228195,-0.546282,-0.913300,-0.551740,-0.759755,-0.632625,0.042814,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,6.0
1,-0.325696,0.015790,-1.697157,0.374641,0.024892,-0.008468,0.409226,-0.219660,-0.661271,0.538649,0.715904,-0.235028,1.199597,-0.468462,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0
2,0.118750,0.355319,1.483022,1.798012,-0.164007,0.977985,-0.188484,-0.155681,1.293542,0.538649,1.561000,0.814425,2.665374,0.554090,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0
3,1.274308,-0.057163,0.757112,-0.247107,0.024892,0.031795,-0.530032,0.420133,0.833586,1.264624,0.293356,-0.235028,0.100264,0.042814,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,0.0
4,0.207639,0.189088,-1.489754,-0.434804,-0.667738,-0.531892,-0.658113,-0.667516,-0.622941,0.538649,-0.551740,-0.759755,-0.266181,0.554090,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,2.0


# Model Definition

## Logistic Regression

First benchmark model. Use standard train & test split and fit the logistic regression.

In [ ]:
# Define the model
model_lr = LogisticRegression()

# Train the model
model_lr.fit(X_train, y_train)

# Predictions
predictions_lr = model_lr.predict(X_test)

## Bayesian Optimization with HYPEROPT

In [ ]:
# Define the Hyperparamters space for Hyperopt
hyperopt_parameters_space = {
    'max_depth': hp.quniform("max_depth", 3, 40, 2),
    'gamma': hp.uniform ('gamma', 1, 15),
    'reg_alpha' : hp.quniform('reg_alpha', 40, 180, 1),
    'reg_lambda' : hp.uniform('reg_lambda', 0, 1),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1),
    'min_child_weight' : hp.quniform('min_child_weight', 0, 60, 3),
    'n_estimators': hp.quniform('n_estimators', 100, 3000, 10),
    'seed': 0
}

In [ ]:
# Define the Objective Function
def objective(space, column_transformer=column_transformer, cv=stratified_kfold, X=X, y=y, scoring='roc_auc'):
    
    # Create the estimator
    clf=xgb.XGBClassifier(objective='binary:logistic',
                          eval_metric='auc',
                          n_estimators=int(space['n_estimators']), 
                          max_depth=int(space['max_depth']), 
                          gamma=space['gamma'],
                          reg_alpha=int(space['reg_alpha']),
                          min_child_weight=int(space['min_child_weight']),
                          colsample_bytree=int(space['colsample_bytree']),
                          use_label_encoder=False)
    
    # Define the Pipeline
    pipeline = Pipeline(steps=[
        ('feature_transformation', column_transformer), 
        ('classifier', clf)
    ])
    
    # Init accuracy and precision list for K-fold
    accuracy = []
    precision = []
    
    # Train the model with K-fold
    for train_index, test_index in cv.split(X, y):
    
        # Train the model
        pipeline.fit(X[train_index], y[train_index])

        # Get predicitons
        predictions = pipeline.predict(X[test_index])
        
        # Calculate accuracy
        accuracy.append(accuracy_score(y[test_index], predictions))
        
        # Calculate precision
        precision.append(precision_score(y[test_index], predictions))
                                  
        
    # Calculate the score
    cv_score = cross_val_score(pipeline, X, y, cv=cv, scoring=scoring, n_jobs=1).mean()
    accuracy_mean = accuracy.mean()
    precision_score = precision.mean()
    
    print(cv_score)
    print(accuracy_mean)
    print(precision_score)
                
    return cv_score

In [ ]:
parameters = fmin(fn=objective,
                  space=hyperopt_parameters_space,
                  algo=tpe.suggest,
                  max_evals=50,
                  trials=Trials())